# Stencil Python Parallel

In [1]:
import sys
print (sys.version)
print (sys.version_info)

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [2]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']


In [7]:
%%writefile spp.py
from mpi4py import MPI
import numpy as np

n            = 4800    # nxn grid (4800,1,500)=1500
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

nsources     = 3       # sources of energy
sources      = np.zeros((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
locnsources  = locx = locy = 0     # number of sources in my area
locsources   = np.zeros((nsources, 2), np.int)  # local to my rank
rheat        = np.zeros(1, np.float64)
size         = n + 2
sizeEnd      = n + 1

comm = MPI.COMM_WORLD
mpirank = comm.rank
mpisize = comm.size
if not mpirank : mpiwtime = -MPI.Wtime()

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(mpisize, 2)
px = pdims[0]
py = pdims[1]
rx = mpirank % px
ry = mpirank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 : north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py : south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 : west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px : east = MPI.PROC_NULL

# decompose the domain   
bx = n // px            # block size in x
by = n // py            # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
        locsources[locnsources, 0] = locx + 2
        locsources[locnsources, 1] = locy + 2
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.float64)
aold = np.zeros((bx+2, by+2), np.float64)

# iterations
niters = (niters + 1) // 2
for iters in range(niters) :
    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()
    # update grid
    anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                        ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                          aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
    # refresh heat sources
    anew[locsources[0:locnsources, 0]-1, locsources[0:locnsources, 1]-1] += energy 

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()
    # update grid
    aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 + 
                        ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                          anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
    # refresh heat sources
    aold[locsources[0:locnsources, 0]-1, locsources[0:locnsources, 1]-1] += energy 
    
# get final heat in the system
comm.Reduce(np.sum(aold[1:bx+1, 1:by+1]), rheat)

# show  
if not mpirank :
    print("Heat=%0.4f | Tempo=%0.4f | MPISize=%0s | Dim=%0s | bx,by=%0s,%0s" 
          %(rheat, mpiwtime+MPI.Wtime(), mpisize, pdims, bx, by))

Overwriting spp.py


In [4]:
%%bash
#2x2=4, 3x3=9, 4x4=16
mpiexec -n 1 python spp.py

Heat=1500.0000 | Tempo=235.7366 | MPISize=1 | Dim=[1, 1] | bx,by=4800,4800


In [9]:
%%bash
mpiexec -n 4 python spp.py

Heat=1500.0000 | Tempo=65.9396 | MPISize=4 | Dim=[2, 2] | bx,by=2400,2400


In [8]:
%%bash
mpiexec -n 9 python spp.py

Heat=1500.0000 | Tempo=36.5017 | MPISize=9 | Dim=[3, 3] | bx,by=1600,1600


In [10]:
%%bash
mpiexec -n 16 python spp.py

Heat=1500.0000 | Tempo=27.2404 | MPISize=16 | Dim=[4, 4] | bx,by=1200,1200


---

Heat=1500.0000 | Tempo=118.0057 | MPISize=4 | Dim=[2, 2] | bx,by=2400,2400


Heat = 1500.0000 | Tempo = 237.5668 s
CPU times: user 2min 27s, sys: 1min 29s, total: 3min 57s
Wall time: 4min 2s


### Gravando em um arquivo no disco

In [36]:
%%writefile stc_pyt_par.py
from mpi4py import MPI
import numpy as np

n            = 4800    # nxn grid (4800,1,500)=1500
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

nsources     = 3       # sources of energy
sources      = np.zeros((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
locnsources  = locx = locy = 0     # number of sources in my area
locsources   = np.zeros((nsources, 2), np.int)  # local to my rank
rheat        = np.zeros(1, np.float64)
size         = n + 2
sizeEnd      = n + 1

comm = MPI.COMM_WORLD
mpirank = comm.rank
mpisize = comm.size
if not mpirank : mpiwtime = -MPI.Wtime()

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(mpisize, 2)
px = pdims[0]
py = pdims[1]
rx = mpirank % px
ry = mpirank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 : north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py : south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 : west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px : east = MPI.PROC_NULL

# decompose the domain   
bx = n // px            # block size in x
by = n // py            # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
#    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
    if(locx >= 0 and locx < bx and locy >= 0 and locy < by) :
        locsources[locnsources, 0] = locx + 2
        locsources[locnsources, 1] = locy + 2
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.float64)
aold = np.zeros((bx+2, by+2), np.float64)

# iterations
niters = (niters + 1) // 2
for iters in range(niters) :
    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()
    # update grid
    anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                        ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                          aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
    # refresh heat sources
    anew[locsources[0:locnsources, 0]-1, locsources[0:locnsources, 1]-1] += energy 

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()
    # update grid
    aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 + 
                        ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                          anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
    # refresh heat sources
    aold[locsources[0:locnsources, 0]-1, locsources[0:locnsources, 1]-1] += energy 
    
# get final heat in the system
comm.Reduce(np.sum(aold[1:bx+1, 1:by+1]), rheat)

# show  
if not mpirank :
    print("Heat=%0.4f | Tempo=%0.4f | MPISize=%0s | Dim=%0s | bx,by=%0s,%0s" 
          %(rheat, mpiwtime+MPI.Wtime(), mpisize, pdims, bx, by))

Overwriting stc_pyt_par.py


In [41]:
%%writefile calctemppar.sh
#!/bin/sh
python stc_pyt_par.py

Overwriting calctemppar.sh


### Testando o funcionamento

In [13]:
%%bash
sh calctemppar.sh

Heat=1500.0000 | Tempo=273.2945 | MPISize=1 | Dim=[1, 1] | bx,by=4800,4800



real	4m41.183s
user	2m40.427s
sys	1m53.247s


In [42]:
%%bash
mpiexec -n 16 python stc_pyt_par.py

Heat=1500.0000 | Tempo=22.3885 | MPISize=16 | Dim=[4, 4] | bx,by=1200,1200


In [ ]:
%%bash
mpiexec -n  python stc_pyt_par.py

### Copia para scratch

In [37]:
%%bash
s='/prj/ampemi/eduardo.miranda2/stnc/Python'
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cp  $s/stc_pyt_par.py $s/spp.py $s/calctemppar.sh $d

### Arquivo de submissão

In [32]:
%%writefile stc_pyt_par_09.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
# 5x5=25 não funciona porque 4800/5 não dá divisão inteira

#SBATCH --ntasks=09            #Total de tarefas
# #SBATCH --nodes=1              #Qtd de nós
# #SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
#SBATCH -J stpypar             #Nome do job, 8 caracteres
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
# #SBATCH --exclusive            #Utilização exclusiva dos nós
#SBATCH --time=00:05:00        #Tempo max. de execução

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
cd /scratch/ampemi/eduardo.miranda2/stnc/Python

#Executavel
EXEC='python stc_pyt_par.py'

#Dispara a execucao
#=======================================#
# ATENÇÃO! precisa do "--mpi=pmi2"      #
#=======================================#
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n ' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting stc_pyt_par_09.srm


## Envia para execução

In [38]:
%%bash
# sbatch stc_pyt_par_01.srm
# sbatch stc_pyt_par_04.srm
sbatch stc_pyt_par_09.srm
sbatch stc_pyt_par_16.srm
sbatch stc_pyt_par_25.srm
# sbatch stc_pyt_par_36.srm
# sbatch stc_pyt_par_49.srm
# sbatch stc_pyt_par_64.srm
# sbatch stc_pyt_par_81.srm

Submitted batch job 774699
Submitted batch job 774700
Submitted batch job 774701


    01: Submitted batch job 774517
    04: Submitted batch job 774641
    09: Submitted batch job 774683
    16: Submitted batch job 774643
    25: Submitted batch job 774644
    36: Submitted batch job 774645
    49: Submitted batch job 774646
    64: Submitted batch job 774647
    81: Submitted batch job 774648

### Verifica se já terminou

In [39]:
%%bash
squeue -n stpypar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


### Mostra os arquivos de saída

In [35]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 5x5=25, 6x6=36, 7x7=49, 8x8=64, 9x9=81
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cat $d/slurm-774517.out    # 01
cat $d/slurm-774641.out    # 04
cat $d/slurm-774683.out    # 09
cat $d/slurm-774643.out    # 16
cat $d/slurm-774644.out    # 25
cat $d/slurm-774645.out    # 36
cat $d/slurm-774646.out    # 49
cat $d/slurm-774647.out    # 64
cat $d/slurm-774648.out    # 81

- Job ID: 774517
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1106
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh calctemppar.sh
Heat=1500.0000 | Tempo=227.9713 | MPISize=1 | Dim=[1, 1] | bx,by=4800,4800

real	3m53.958s
user	2m59.150s
sys	0m49.200s
-- FIM --------------------------------
- Job ID: 774641
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1064
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  4 python stc_pyt_par.py
Heat=1500.0000 | Tempo=64.1276 | MPISize=4 | Dim=[2, 2] | bx,by=2400,2400
-- FIM --------------------------------
- Job ID: 774683
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1286
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.mira

In [40]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 5x5=25, 6x6=36, 7x7=49, 8x8=64, 9x9=81
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cat $d/slurm-774699.out
cat $d/slurm-774700.out
cat $d/slurm-774701.out

- Job ID: 774699
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1101
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  9 python stc_pyt_par.py
Heat=1500.0000 | Tempo=43.2708 | MPISize=9 | Dim=[3, 3] | bx,by=1600,1600
-- FIM --------------------------------
- Job ID: 774700
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 16
- Nos alocados: sdumont1376
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  16 python stc_pyt_par.py
Heat=1500.0000 | Tempo=33.5772 | MPISize=16 | Dim=[4, 4] | bx,by=1200,1200
-- FIM --------------------------------
- Job ID: 774701
- Tarefas por no:
- Qtd. de nos: 2
- Tot. de tarefas: 25
- Nos alocados: sdumont[1081-1082]
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun ------

# Segunda tomada de tempo

In [1]:
%%bash
sbatch stc_pyt_par_01.srm
sbatch stc_pyt_par_04.srm
sbatch stc_pyt_par_09.srm
sbatch stc_pyt_par_16.srm
sbatch stc_pyt_par_36.srm
sbatch stc_pyt_par_49.srm
sbatch stc_pyt_par_64.srm
sbatch stc_pyt_par_81.srm

Submitted batch job 788064
Submitted batch job 788065
Submitted batch job 788066
Submitted batch job 788067
Submitted batch job 788068
Submitted batch job 788069
Submitted batch job 788070
Submitted batch job 788071


In [2]:
! squeue -n stpypar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788064 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788066 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788067 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788068 cpu_small  stpypar eduardo. PD       0:00      2 (Priority)
            788069 cpu_small  stpypar eduardo. PD       0:00      3 (Priority)
            788070 cpu_small  stpypar eduardo. PD       0:00      3 (Priority)
            788071 cpu_small  stpypar eduardo. PD       0:00      4 (Priority)
            788065   cpu_dev  stpypar eduardo.  R       0:04      1 sdumont1065


In [3]:
! squeue -n stpypar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [4]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cat $d/slurm-788064.out    # 01
cat $d/slurm-788065.out    # 04
cat $d/slurm-788066.out    # 09
cat $d/slurm-788067.out    # 16
cat $d/slurm-788068.out    # 36
cat $d/slurm-788069.out    # 49
cat $d/slurm-788070.out    # 64
cat $d/slurm-788071.out    # 81

- Job ID: 788064
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh calctemppar.sh
Heat=1500.0000 | Tempo=226.2890 | MPISize=1 | Dim=[1, 1] | bx,by=4800,4800
-- FIM --------------------------------
- Job ID: 788065
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1065
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  4 python stc_pyt_par.py
Heat=1500.0000 | Tempo=66.1329 | MPISize=4 | Dim=[2, 2] | bx,by=2400,2400
-- FIM --------------------------------
- Job ID: 788066
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun --------------------

# Terceira tomada de tempo

In [9]:
%%bash
sbatch stc_pyt_par_01.srm
sbatch stc_pyt_par_04.srm
sbatch stc_pyt_par_09.srm
sbatch stc_pyt_par_16.srm
sbatch stc_pyt_par_36.srm
sbatch stc_pyt_par_49.srm
sbatch stc_pyt_par_64.srm
sbatch stc_pyt_par_81.srm

Submitted batch job 788075
Submitted batch job 788076
Submitted batch job 788077
Submitted batch job 788078
Submitted batch job 788079
Submitted batch job 788080
Submitted batch job 788081
Submitted batch job 788082


In [10]:
! squeue -n stpypar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788070 cpu_small  stpypar eduardo. PD       0:00      3 (Resources)
            788071 cpu_small  stpypar eduardo. PD       0:00      4 (Resources)
            788068 cpu_small  stpypar eduardo. PD       0:00      2 (Resources)
            788069 cpu_small  stpypar eduardo. PD       0:00      3 (Resources)
            788067 cpu_small  stpypar eduardo. PD       0:00      1 (Resources)
            788066 cpu_small  stpypar eduardo. PD       0:00      1 (Resources)
            788064 cpu_small  stpypar eduardo. PD       0:00      1 (Resources)
            788075 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788077 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788078 cpu_small  stpypar eduardo. PD       0:00      1 (Priority)
            788079 cpu_small  stpypar eduardo. PD       0:00      2 (Priority)
            788080 cpu_small  stpypar e

In [1]:
! squeue -n stpypar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [2]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cat $d/slurm-788075.out    # 01
cat $d/slurm-788076.out    # 04
cat $d/slurm-788077.out    # 09
cat $d/slurm-788078.out    # 16
cat $d/slurm-788079.out    # 36
cat $d/slurm-788080.out    # 49
cat $d/slurm-788081.out    # 64
cat $d/slurm-788082.out    # 81

- Job ID: 788075
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh calctemppar.sh
Heat=1500.0000 | Tempo=227.2968 | MPISize=1 | Dim=[1, 1] | bx,by=4800,4800
-- FIM --------------------------------
- Job ID: 788076
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1064
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  4 python stc_pyt_par.py
Heat=1500.0000 | Tempo=63.9705 | MPISize=4 | Dim=[2, 2] | bx,by=2400,2400
-- FIM --------------------------------
- Job ID: 788077
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun --------------------

In [11]:
! squeue -u eduardo.miranda2 | grep -c ^

48
